# ITINERAIRE DE VACANCE - EXPLORATION DES DONNÉES

## 0. Import des modules

In [1]:
import pandas as pd
import json
from pprint import pprint
import seaborn as sns
import geopandas
import contextily as cx
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import folium

## 1. Import des données
### 1.1 JSON
#### 1.1.1 Index

In [2]:
objects_path = '../Data/JSON/objects/'
index_path = '../Data/JSON/index.json'

with open(index_path) as json_data:
    index = json.load(json_data)

L'index contient plusieurs informations:
* `file`: chemin d'accès vers les objets json
* `label`: nom du Point Of Interrest
* `lastUpdateDatatourisme`: dernière mise à jour du POI sur Datatourisme

In [4]:
pprint(index[:2])

[{'file': '0/00/13-000fac17-cff9-3918-a3d6-8edc919c8533.json',
  'label': 'Calvi Jet Locations',
  'lastUpdateDatatourisme': '2023-02-22T06:42:37.696Z'},
 {'file': '0/01/13-016f3145-67f8-33c3-9293-109b420a9104.json',
  'label': 'Promenades du Golfe',
  'lastUpdateDatatourisme': '2023-04-02T05:48:09.233Z'}]


#### 1.1.2 Exploration d'un objet json

In [5]:
jsonObject_path = objects_path + index[0]['file']

with open(jsonObject_path) as json_data:
    jsonObject = json.load(json_data)

Nous avons 15 clés dans notre objet json:
* **`@id`: Lien vers le POI.**
* `dc:identifier`: Identifiant unique
* **`@type`: La classification de POI. Par exemple : festival, musée, chambre d’hôtes...**
* `rdfs:comment`: Commentaire du POI
* **`rdfs:label`: Nom du Point Of Interrest**
* `hasBeenCreatedBy`: L'agent qui a créé ce POI dans le système d'information :
    * `@id`: Lien vers l'agent sur DATAtourisme
    * `dc:identifier`: Identifiant unique
    * `schema:email`: E-mail de l'agent
    * `schema:legalName`: Nom officiel de l'agent
    * `@type`: Type de l'agent
    * `foaf:homepage`: Lien vers le site de l'agent
* `hasBeenPublishedBy`: L'agent qui a publié ce POI dans le système d'information:
    * `@id`: Lien vers l'agent sur DATAtourisme
    * `schema:legalName`: Nom officiel de l'agent
    * `@type`: Type de l'agent
    * `foaf:homepage`: Lien vers le site de l'agent
* **`hasContact`: L'agent à contacter pour affaires générales relatives à ce POI:**
    * `@id`: Lien vers l'agent sur DATAtourisme
    * `schema:email`: E-mail de l'agent
    * **`schema:telephone`: Telephone de l'agent**
    * `@type`: Type de l'agent
    * **`foaf:homepage`: Lien vers le site de l'agent**
* **`hasDescription`: Description textuelle courte ou longue du POI pouvant être associée à une audience. Par exemple, un POI peut avoir une description dédiée aux écoles et une autre dédiée au grand public. Si une description n'a aucune audience renseignée, on suppose qu'elle est dédiée à tous les publics:**
    * `@id`: Lien vers la description
    * `@type`: Type(s) de la descripton
    * `hasTranslatedProperty`: Informations de traductions:
        * `@id`: Lien vers les informations de la traduction de la description
        * `dc:contributor`: Traducteur (api.deepl.com/v2/translate)
        * `rdf:language`: Langue de la traduction
        * `rdf:predicate`: Type de la description
    * **`shortDescription`: Dictionnaire des descriptions traduit en plusieurs langues:**
        * `fr`: En fr
        * `de`: ...
* **`hasTheme`: Liste des thèmes:**
    * `@id`: Identifiant du thème au format kb:[theme en anglais sans espace]
    * `@type`: Type du thème
    * `rdfs:label`: Thème en différente langue:
        * `fr`: fr
        * `de`: ...
* `hasTranslatedProperty`: liste d'informations de traductions sur le commentaire:
    * `@id`: Lien vers les informations de la traduction du commentaire
    * `dc:contributor`: Traducteur (api.deepl.com/v2/translate)
    * `rdf:language`: Langue de la traduction
    * `rdf:predicate`: Table de jointure entre @id et comment
* `isLocatedAt`: Informations sur l'emplacement géographique du POI:
    * `@id`: Lien vers les informations sur l'emplacement
    * `schema:address`: Informations sur l'adresse postale
        * `@id`: Lien vers les informations de l'adresse postale
        * **`schema:addressLocality`: Ville**
        * **`schema:postalCode`: Code postale**
        * **`schema:streetAddress`: Adresse postale (certaines données ne sont pas propres)**
        * `@type`: Type de l'adresse
        * `hasAddressCity`: Informations complémentaires sur l'adresse postale
            * `@id`: Identifiant
            * `@type`: Type de localisation (ex: City)
            * **`rdfs:label`: Liste de la ville traduite ['fr']**
            * `insee`: Code Insee
            * `isPartOfDepartment`: Informations du département
                * `@id`: Identifiant
                * `@type`: Type de localisation (ex: Department)
                * **`rdfs:label`: Liste du Department traduite ['fr']**
                * `insee`: Code Insee
                * `isPartOfRegion`: Informations du département:
                    * `@id`: Identifiant
                    * `@type`: Type de localisation (ex: Region)
                    * **`rdfs:label`: Liste de la Region traduite ['fr']**
                    * `insee`: Code Insee
                    * `isPartOfCountry`: Informations du Pays:
                        * `@id`: Identifiant
                        * `@type`: Type de localisation (ex: Country)
                        * **`rdfs:label`: Liste Du Pays traduite ['fr']**
    * `schema:geo`:
        * `@id`: Lien vers les informations coordonnées gépgraphiques
        * **`schema:latitude`: Latitude**
        * **`schema:longitude`: Longitude**
        * `@type`: Type
    * `schema:openingHoursSpecification`: Informations des horraires d'ouverture et fermeture 
        * `@id`: Lien vers les informations des horraires
        * **`schema:closes`: Heure de fermeture**
        * **`schema:opens`: Heure d'ouverture**
        * **`schema:validFrom`: Date de début de période d'ouverture**
        * **`schema:validThrough`: Date de fin de période d'ouverture**
        * `@type`: Type
        * `additionalInformation`: Liste d'inforamtions supplémentaires traduit en plusieurs langues
        * `hasTranslatedProperty`: Liste des informations de traduction des infos supplémentaires
            * `@id`: Lien vers les infos de trad
            * `dc:contributor`: Traducteur (api.deepl.com/v2/translate)
            * `rdf:language`: Langue de la traduction
            * `rdf:predicate`: Table de jointure entre @id et additionalInformation
    * `@type`: Type de place
* `isOwnedBy`: Informations de l'agent propriétaire
    * `@id`: Lien vers l'agent sur DATAtourisme
    * `schema:email`: E-mail de l'agent
    * `schema:legalName`: Nom de l'agent
    * `@type`: Type de l'agent
    * `foaf:homepage`: Lien vers le site de l'agent    
* **`lastUpdate`: Dernière mise à jour du POI**
* `lastUpdateDatatourisme`: Dernière mise à jour du POI sur DATAtourisme

In [6]:
# pprint(jsonObject)

## 2. Préparation des données

### 2.1 Sélection des colonnes et Aplatissement du json

In [17]:
def dataframeFromJson(objects_path, index_path):
    listOfPathJson = []
    
    for obj in index:
        listOfPathJson.append(objects_path + obj['file'])
        
    listOfJson = []
    
    for json_path in listOfPathJson:
        with open(json_path) as json_data:
            jsonObject = json.load(json_data)
            listOfJson.append(jsonObject)
    
    col_select_raw = [
    '@id',
    'rdfs:label.fr',
    '@type',
    'hasContact',
    'hasDescription',
    'hasTheme',
    'isLocatedAt',
    'lastUpdate'
    ]
    
    df = pd.json_normalize(listOfJson, errors='ignore')[col_select_raw]
    
    clean_name = {
        '@id': 'id',
        'rdfs:label.fr': 'nom',
        '@type': 'type',
        'hasContact': 'contact',
        'hasDescription': 'description',
        'hasTheme': 'theme',
        'isLocatedAt': 'location',
        'lastUpdate': 'maj'
    }
    
    return df.rename(columns=clean_name)

In [18]:
df_raw = dataframeFromJson(objects_path, index_path)
df_raw.head(2)

,id,nom,type,contact,description,theme,location,maj
0,https://data.datatourisme.fr/13/000fac17-cff9-...,[Calvi Jet Locations],"[schema:LocalBusiness, ActivityProvider, Place...",[{'@id': 'https://data.datatourisme.fr/d7f7f12...,[{'@id': 'https://data.datatourisme.fr/1759cd3...,"[{'@id': 'kb:SeaKayaking', '@type': ['SportsTh...",[{'@id': 'https://data.datatourisme.fr/f4ee540...,2022-08-11
1,https://data.datatourisme.fr/13/016f3145-67f8-...,[Promenades du Golfe],"[schema:LocalBusiness, PlaceOfInterest, PointO...",[{'@id': 'https://data.datatourisme.fr/ffcc278...,[{'@id': 'https://data.datatourisme.fr/b120860...,"[{'@id': 'kb:Fishing', '@type': ['SportsTheme'...",[{'@id': 'https://data.datatourisme.fr/91088f9...,2023-04-01


### 2.2 Preprocessing des données
#### 2.2.1 Type

In [194]:
def preprocessData(df):
    df_preprocessed = df.copy()
    
    # Nom
    df_preprocessed['nom'] = df_preprocessed['nom'].map(lambda x: x[0])
    
    # Type
    df_preprocessed['type'] = df_preprocessed['type'].map(lambda x: x[-1])
    
    # Contact
    df_preprocessed['contactId'] = df_preprocessed['contact']\
    .map(lambda x: x[0]['@id'] if type(x) != float else None)
    
    df_contact = pd.DataFrame()
    
    for el in df_raw['contact']:
        if type(el) != float:
            df_contact = pd.concat([df_contact, pd.json_normalize(el[0])])
    
    df_contact.set_index("@id", inplace=True)
    
    df_preprocessed = df_preprocessed.join(df_contact, on='contactId', how='left') \
               .drop_duplicates(subset='id') \
               .drop(columns=['contactId', 'contact', '@type'])
    
    df_preprocessed = df_preprocessed.rename(columns={
        'schema:email': 'email',
        'schema:telephone': 'telephone',
        'foaf:homepage': 'homepage'
    })
    
    df_preprocessed[['email', 'telephone', 'homepage']] = \
    df_preprocessed[['email', 'telephone', 'homepage']]\
    .applymap(lambda x: x[0] if type(x) != float else None)
    
    # Description
    df_preprocessed['description'] = \
    df_preprocessed['description']\
    .map(lambda x: x[0]["shortDescription"]["fr"][0] if type(x) != float else None)
    
    # Theme unique
    df_theme = pd.DataFrame()
    
    for el in df_preprocessed['theme']:
        if type(el) != float:
            df_theme = pd.concat([df_theme, pd.json_normalize(el)]).drop_duplicates(['@id'])
            
    df_theme_unique = df_theme.set_index("@id")[['@type', 'rdfs:label.fr']]
    df_theme_unique = df_theme_unique.rename(columns={
        '@type': 'typeTheme',
        'rdfs:label.fr': 'nomTheme'
    })
    df_theme_unique[['typeTheme', 'nomTheme']] = \
    df_theme_unique[['typeTheme', 'nomTheme']]\
    .applymap(lambda x: x[0] if type(x) != float else None)
    
    # Theme
    df_theme = df_preprocessed[['id', 'theme']].copy()
    
    df_theme['theme'] = df_theme['theme']\
    .map(lambda x: [el['@id'] for el in x] if type(x) != float else None)
    
    df_theme = df_theme.explode('theme')
    
    # Location
    df_location = pd.DataFrame()
    
    for el in df_preprocessed['location']:
        if type(el) != float:
            df_location = pd.concat([df_location, pd.json_normalize(el[0])])
    
    df_location = df_location.set_index("@id")
    
    d ={"id_adresse": [],
    "Ville": [],
    "CodePostale": [],
    "Departement": [],
    "Region": []}

    for el in df_location['schema:address']:
        id_adresse = el[0]['@id']
        ville = el[0]['schema:addressLocality']
        cp = el[0]['schema:postalCode']
        dep = el[0]['hasAddressCity']['isPartOfDepartment']['rdfs:label']['fr']
        reg = el[0]['hasAddressCity']['isPartOfDepartment']['isPartOfRegion']['rdfs:label']['fr']
        d['id_adresse'].append(id_adresse)
        d['Ville'].append(ville)
        d['CodePostale'].append(cp)
        d['Departement'].append(dep[0])
        d['Region'].append(reg[0])
        
    df_adresse = pd.DataFrame(d).set_index("id_adresse")
    df_location['id_adresse'] = df_location['schema:address'].map(lambda x: x[0]['@id'])
    df_location = df_location.join(df_adresse, how='left', on='id_adresse')
    
    d ={"id_opening": [],
    "Close": [],
    "Open": [],
    "validFrom": [],
    "validThrough": []}

    for el in df_location['schema:openingHoursSpecification']:
        id_opening = el[0]['@id']
        try:
            Close = el[0]['schema:closes']
        except:
            Close = None
        try:
            Open = el[0]['schema:opens']
        except:
            Open = None
        try:
            validFrom = el[0]['schema:validFrom']
        except:
            validFrom = None
        try:
            validThrough = el[0]['schema:validThrough']
        except:
            validThrough = None
        d['id_opening'].append(id_opening)
        d['Close'].append(Close)
        d['Open'].append(Open)
        d['validFrom'].append(validFrom)
        d['validThrough'].append(validThrough)
        
    df_opening = pd.DataFrame(d).set_index("id_opening")
    df_location['id_opening'] = df_location['schema:openingHoursSpecification'].map(lambda x: x[0]['@id'])
    df_location = df_location.join(df_opening, how='left', on='id_opening')[[
    "schema:geo.schema:latitude",
    "schema:geo.schema:longitude",
    "Ville",
    "CodePostale",
    "Departement",
    "Region",
    "Close",
    "Open",
    "validFrom",
    "validThrough",
]]
    
    df_preprocessed['location'] = df_preprocessed['location'].map(lambda x: x[0]['@id'])
    df_preprocessed = df_preprocessed.join(df_location, on='location', how='left') \
               .drop_duplicates(subset='id') \
               .drop(columns=['location'])
    
    df_preprocessed = df_preprocessed.rename(columns={
    'schema:geo.schema:latitude': 'latitude',
    'schema:geo.schema:longitude': 'longitude'
})
    
    df_preprocessed[['latitude', 'longitude']] = \
    df_preprocessed[['latitude', 'longitude']].astype(float)
    
    return df_preprocessed, df_theme_unique, df_theme

In [195]:
df_preprocessed, df_theme_unique, df_theme = preprocessData(df_raw)

In [197]:
df_preprocessed

,id,nom,type,description,theme,maj,email,telephone,homepage,latitude,longitude,Ville,CodePostale,Departement,Region,Close,Open,validFrom,validThrough
0,https://data.datatourisme.fr/13/000fac17-cff9-...,Calvi Jet Locations,Store,"Lors de vos vacances en corse, les amateurs de...","[{'@id': 'kb:SeaKayaking', '@type': ['SportsTh...",2022-08-11,calvijet@gmail.com,+33 6 26 17 04 97,https://www.calvi-jet.fr/,42.555104,8.761533,Calvi,20260,Haute-Corse,Corse,20:30:00,08:00:00,2023-05-01T00:00:00,2023-10-31T23:59:59
1,https://data.datatourisme.fr/13/016f3145-67f8-...,Promenades du Golfe,SportsAndLeisurePlace,Promenades au rythme d'antan à bord d'Ulysse c...,"[{'@id': 'kb:Fishing', '@type': ['SportsTheme'...",2023-04-01,promenadesdugolfe.ulysse@gmail.com,+33 6 15 07 34 68,https://www.promenadesdugolfe.com/,41.590132,9.283183,Porto-Vecchio,20137,Corse-du-Sud,Corse,None,None,2023-01-01T00:00:00,2023-12-31T23:59:59
2,https://data.datatourisme.fr/13/018c05e0-d6c7-...,Bureau d'Information Touristique de la Citadelle,TouristInformationCenter,"En saison, le Bureau d'Information Touristique...",NaN,2022-07-06,tourisme@balagne-corsica.com,+33 4 95 65 16 67,https://www.balagne-corsica.com/,42.568000,8.759500,Calvi,20260,Haute-Corse,Corse,None,None,2023-06-06T00:00:00,2023-09-30T23:59:59
3,https://data.datatourisme.fr/13/019b2930-944c-...,U casteddu d’Araghju,PointOfInterest,"Forteresse cyclopéenne, aux remparts démesurés...","[{'@id': 'kb:Culture', '@type': ['CulturalThem...",2022-05-20,None,+33 4 95 70 09 58,None,41.656311,9.279691,San-Gavino-di-Carbini,20170,Corse-du-Sud,Corse,None,None,2023-01-01T00:00:00,2023-12-31T23:59:59
4,https://data.datatourisme.fr/13/0278689c-4ed8-...,Belvédère de Palombaggia,PointOfInterest,"A 900 mètres de la plage de Palombaggia, résid...",NaN,2023-02-06,infopalombaggia@gmail.com,+33 4 95 70 95 70,http://www.palombaggia.com/,41.564000,9.319300,Porto-Vecchio,20137,Corse-du-Sud,Corse,None,None,2023-04-07T00:00:00,2023-11-05T23:59:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,https://data.datatourisme.fr/13/fe353538-c261-...,Centre National de Création Musicale Voce,SportsAndLeisurePlace,"Axé sur les musiques(traditionnelles, contempo...",NaN,2023-01-23,info@voce.corsica,+33 4 95 61 73 13,http://www.voce.corsica/,42.599720,8.902970,Pigna,20220,Haute-Corse,Corse,None,None,2023-01-01T00:00:00,2023-12-31T23:59:59
619,https://data.datatourisme.fr/13/fee026aa-0aa1-...,Plage de Ghjunchitu et Bodri,SportsAndLeisurePlace,Venez profiter des plages de Bodri et de Ghjun...,"[{'@id': 'kb:Bathing', '@type': ['Accommodatio...",2022-09-28,None,None,None,42.627000,8.905800,Corbara,20220,Haute-Corse,Corse,None,None,2023-01-01T00:00:00,2023-12-31T23:59:59
620,https://data.datatourisme.fr/13/ff22bb83-6515-...,A Funtana,PointOfInterest,"Lors de votre séjour en Corse, faites une paus...","[{'@id': 'kb:MediterraneanCuisine', '@type': [...",2023-02-28,contact@afuntana.fr,+33 4 95 65 09 52,http://www.afuntana.fr/,42.566999,8.757473,Calvi,20260,Haute-Corse,Corse,None,None,2023-03-01T00:00:00,2023-11-02T23:59:59
621,https://data.datatourisme.fr/13/ffc6590c-e617-...,Le Bistrot Via Marine,PointOfInterest,Pause gourmande en Corse ... La Cuisine by Via...,"[{'@id': 'kb:MediterraneanCuisine', '@type': [...",2022-05-03,None,+33 4 95 65 06 30,None,42.566000,8.758100,Calvi,20260,Haute-Corse,Corse,None,None,2023-04-01T00:00:00,2023-10-31T23:59:59


In [65]:
df_POI = df_raw.copy()

In [100]:
df_POI['contactId'] = df_raw['contact'].map(lambda x: x[0]['@id'] if type(x) != float else None)

In [116]:
df_contact = pd.DataFrame()
for el in df_raw['contact']:
    if type(el) != float:
        df_contact = pd.concat([df_contact, pd.json_normalize(el[0])])
#         df_POI = df_POI.join(pd.json_normalize(el[0]).set_index("@id"), on='contactId', how='left').drop_duplicates(subset='@id')
df_contact.set_index("@id", inplace=True)
df_contact[['schema:email']]

,schema:email,schema:telephone,@type,foaf:homepage
@id,,,,
https://data.datatourisme.fr/d7f7f123-e07f-3119-96de-88ac75239c71,[calvijet@gmail.com],[+33 6 26 17 04 97],"[foaf:Agent, Agent]",[https://www.calvi-jet.fr/]
https://data.datatourisme.fr/ffcc2787-f836-3c5e-9bfa-d947c7089ae7,[promenadesdugolfe.ulysse@gmail.com],"[+33 6 15 07 34 68, +33 6 21 09 21 41]","[foaf:Agent, Agent]",[https://www.promenadesdugolfe.com/]
https://data.datatourisme.fr/8943b355-5c02-333a-815b-a44aa97d99d1,[tourisme@balagne-corsica.com],[+33 4 95 65 16 67],"[foaf:Agent, Agent]",[https://www.balagne-corsica.com/]
https://data.datatourisme.fr/4c728769-0f7b-3af4-927c-1364b54b762d,NaN,[+33 4 95 70 09 58],"[foaf:Agent, Agent]",NaN
https://data.datatourisme.fr/6d5b8580-42f1-3ec7-8981-6ccb8f1da905,[infopalombaggia@gmail.com],[+33 4 95 70 95 70],"[foaf:Agent, Agent]",[http://www.palombaggia.com/]
...,...,...,...,...
https://data.datatourisme.fr/363972cc-23a5-3785-b74f-d69626170abe,[panoramic@vacances-andretrigano.com],"[+33 4 95 60 73 13, +33 5 61 69 05 07]","[foaf:Agent, Agent]",[https://vacances-andretrigano.com/camping/hau...
https://data.datatourisme.fr/8bdc849a-057c-3774-b6f6-46e070400896,[info@voce.corsica],"[+33 4 95 61 73 13, +33 6 79 40 68 80]","[foaf:Agent, Agent]",[http://www.voce.corsica/]
https://data.datatourisme.fr/be58956d-3f28-366f-9540-4779179cd77d,[contact@afuntana.fr],[+33 4 95 65 09 52],"[foaf:Agent, Agent]",[http://www.afuntana.fr/]


In [82]:
pd.concat([df_raw, df_raw['contact'].apply(lambda x: pd.json_normalize(x[0]) if type(x) != float else None)])

,id,nom,type,contact,description,theme,location,maj,0
0,https://data.datatourisme.fr/13/000fac17-cff9-...,[Calvi Jet Locations],"[schema:LocalBusiness, ActivityProvider, Place...",[{'@id': 'https://data.datatourisme.fr/d7f7f12...,[{'@id': 'https://data.datatourisme.fr/1759cd3...,"[{'@id': 'kb:SeaKayaking', '@type': ['SportsTh...",[{'@id': 'https://data.datatourisme.fr/f4ee540...,2022-08-11,NaN
1,https://data.datatourisme.fr/13/016f3145-67f8-...,[Promenades du Golfe],"[schema:LocalBusiness, PlaceOfInterest, PointO...",[{'@id': 'https://data.datatourisme.fr/ffcc278...,[{'@id': 'https://data.datatourisme.fr/b120860...,"[{'@id': 'kb:Fishing', '@type': ['SportsTheme'...",[{'@id': 'https://data.datatourisme.fr/91088f9...,2023-04-01,NaN
2,https://data.datatourisme.fr/13/018c05e0-d6c7-...,[Bureau d'Information Touristique de la Citade...,"[schema:LocalBusiness, schema:TouristInformati...",[{'@id': 'https://data.datatourisme.fr/8943b35...,[{'@id': 'https://data.datatourisme.fr/a240758...,NaN,[{'@id': 'https://data.datatourisme.fr/a768f98...,2022-07-06,NaN
3,https://data.datatourisme.fr/13/019b2930-944c-...,[U casteddu d’Araghju],"[ArcheologicalSite, CulturalSite, PlaceOfInter...",[{'@id': 'https://data.datatourisme.fr/4c72876...,[{'@id': 'https://data.datatourisme.fr/9164409...,"[{'@id': 'kb:Culture', '@type': ['CulturalThem...",[{'@id': 'https://data.datatourisme.fr/a409144...,2022-05-20,NaN
4,https://data.datatourisme.fr/13/0278689c-4ed8-...,[Belvédère de Palombaggia],"[schema:Accommodation, schema:LodgingBusiness,...",[{'@id': 'https://data.datatourisme.fr/6d5b858...,[{'@id': 'https://data.datatourisme.fr/6a7692f...,NaN,[{'@id': 'https://data.datatourisme.fr/e64583b...,2023-02-06,NaN
...,...,...,...,...,...,...,...,...,...
618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...
619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...
621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...


In [92]:
df_raw['contact'].map(lambda x: pd.json_normalize(x[0]) if type(x) != float else None)[0]

,@id,schema:email,schema:telephone,@type,foaf:homepage
0,https://data.datatourisme.fr/d7f7f123-e07f-311...,[calvijet@gmail.com],[+33 6 26 17 04 97],"[foaf:Agent, Agent]",[https://www.calvi-jet.fr/]


In [67]:
df_POI = df_POI.join(df_contact, on='contact', how='left') \
               .drop_duplicates(subset='@id') \
               .drop(columns=['contact'])
df_POI

TypeError: unhashable type: 'list'

In [109]:
pd.json_normalize(df_raw['contact'].iloc[0])

,@id,schema:email,schema:telephone,@type,foaf:homepage
0,https://data.datatourisme.fr/d7f7f123-e07f-311...,[calvijet@gmail.com],[+33 6 26 17 04 97],"[foaf:Agent, Agent]",[https://www.calvi-jet.fr/]


In [149]:
df_POI['theme'][0]

[{'@id': 'kb:SeaKayaking',
  '@type': ['SportsTheme'],
  'rdfs:label': {'de': ['Seekajak'],
   'pt': ['Caiaque marítimo'],
   'en': ['Sea kayaking'],
   'it': ['Kayak da mare'],
   'fr': ['Kayak de mer'],
   'nl': ['Zeekajakken'],
   'es': ['Kayak de mar']}},
 {'@id': 'kb:StandupPaddleboarding',
  '@type': ['SportsTheme'],
  'rdfs:label': {'de': ['Stand-up Paddle (SUP)'],
   'pt': ['Paddle de pé (SUP)'],
   'en': ['Standup paddleboarding'],
   'it': ['Stand up paddle (SUP)'],
   'fr': ['Stand up paddle sup'],
   'es': ['Tabla de pádel surf'],
   'nl': ['Suppen']}},
 {'@id': 'kb:TowableTube',
  '@type': ['SportsTheme'],
  'rdfs:label': {'de': ['Banana Boat'],
   'pt': ['Boia de tração'],
   'en': ['Towable tube'],
   'it': ['Ciambella trainata'],
   'fr': ['Bouée tractée'],
   'nl': ['Tubing'],
   'es': ['Skibus náutico']}},
 {'@id': 'kb:Wakeboarding',
  '@type': ['SportsTheme'],
  'rdfs:label': {'de': ['Wake Board'],
   'pt': ['Wakeboard'],
   'en': ['Wakeboarding'],
   'it': ['Wakeboa